In [2]:
from vosk import Model, KaldiRecognizer
import wave
import json
import os

AUDIO_FILES = ["trump1.wav", "trump2.wav", "trump3.wav"]
MODEL_PATH = "./vosk-model-small-en-us-0.15"

model = Model(MODEL_PATH)

def transcribe(audio_path):
    wf = wave.open(audio_path, "rb")
    rec = KaldiRecognizer(model, wf.getframerate())
    transcription = ""

    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            res = json.loads(rec.Result())
            transcription += " " + res.get("text", "")
    transcription += " " + json.loads(rec.FinalResult()).get("text", "")
    return transcription.strip()

for audio in AUDIO_FILES:
    print(f"🔊 Transcribiendo {audio}...")
    text = transcribe(audio)
    out_path = audio.replace(".wav", ".txt")
    with open(out_path, "w") as f:
        f.write(text)
    print(f"✅ Guardado en {out_path}")


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from ./vosk-model-small-en-us-0.15/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from ./vosk-model-small-en-us-0.15/graph/HCLr.fst ./vosk-model-small-en-us-0.15/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:303) Loading winfo ./vosk-model-small-en-us-0.15/graph/phones/word_boundary.int


🔊 Transcribiendo trump1.wav...
✅ Guardado en trump1.txt
🔊 Transcribiendo trump2.wav...
✅ Guardado en trump2.txt
🔊 Transcribiendo trump3.wav...
✅ Guardado en trump3.txt


In [12]:
import pickle
import numpy as np
from sklearn.linear_model import PoissonRegressor, LinearRegression

# ----------------------
# 1. Definición de palabras objetivo
# ----------------------
PALABRAS_OBJETIVO = ["china", "economy", "border"]

# ----------------------
# 2. Función para cargar y tokenizar un .txt
# ----------------------
def cargar_palabras(ruta_txt):
    """
    Lee archivo .txt, convierte a minúsculas, elimina saltos de línea y signos de puntuación básicos,
    y retorna lista de palabras tokenizadas.
    """
    with open(ruta_txt, "r", encoding="utf-8") as f:
        texto = f.read().lower()
    for ch in ['.', ',', ';', ':', '!', '?', '"', '“', '”', '(', ')', '[', ']', '—', '…']:
        texto = texto.replace(ch, "")
    palabras = texto.split()
    return palabras

# ----------------------
# 3. Función para precalcular posiciones de cada palabra objetivo
# ----------------------
def posiciones_por_palabra(palabras, palabras_objetivo):
    """
    Dada la lista completa de 'palabras' y la lista 'palabras_objetivo',
    devuelve un diccionario:
      {
        w1: [pos1, pos2, ...],  # índices donde w1 aparece
        w2: [pos1, pos2, ...],
        ...
      }
    """
    posiciones = { w: [] for w in palabras_objetivo }
    for idx, w in enumerate(palabras):
        if w in posiciones:
            posiciones[w].append(idx)
    return posiciones

# ----------------------
# 4. Función para generar dataset para cada palabra objetivo (regresión de conteo)
# ----------------------
def generar_dataset_multi(palabras, palabras_objetivo):
    """
    Construye X e Y para entrenamiento, retornando:
      - X: array de shape (N, 8) con features para cada posición i
      - Y_reg: dict de arrays con conteo exacto de apariciones futuras para cada palabra objetivo
    Cada X[i] es un vector:
      [ i, count_china, count_economy, count_border, freq_china, freq_economy, freq_border, position_ratio ]
    Y_reg[w][i] = número exacto de veces que w aparece en posiciones > i.
    """
    N = len(palabras)
    pos = posiciones_por_palabra(palabras, palabras_objetivo)

    # Inicializar diccionario de resultados
    Y_reg = { w: [] for w in palabras_objetivo }
    X = []

    # Contadores acumulados hasta i-1
    count_so_far = { w: 0 for w in palabras_objetivo }

    for i, w_actual in enumerate(palabras):
        # Calcular remaining_count para cada palabra objetivo
        remaining_count = {}
        for w in palabras_objetivo:
            # Cantidad de posiciones en pos[w] que sean > i
            remaining_count[w] = sum(1 for p in pos[w] if p > i)
            Y_reg[w].append(remaining_count[w])

        # Features “lag” (antes de procesar palabra actual)
        count_until_prev = { w: count_so_far[w] for w in palabras_objetivo }
        freq_until_prev = {
            w: (count_until_prev[w] / (i + 1)) if (i + 1) > 0 else 0.0
            for w in palabras_objetivo
        }
        position_ratio = (i + 1) / N

        # Vector de features
        feature_i = [
            i,
            count_until_prev["china"],
            count_until_prev["economy"],
            count_until_prev["border"],
            freq_until_prev["china"],
            freq_until_prev["economy"],
            freq_until_prev["border"],
            position_ratio
        ]
        X.append(feature_i)

        # Actualizar count_so_far para la siguiente iteración
        if w_actual in count_so_far:
            count_so_far[w_actual] += 1

    return np.array(X), { w: np.array(Y_reg[w]) for w in palabras_objetivo }

# ----------------------
# 5. Cargar datos de entrenamiento y generar X_train, Y_train_reg
# ----------------------
palabras1 = cargar_palabras("trump1.txt")
palabras3 = cargar_palabras("trump3.txt")

# Concatenamos para aumentar muestras
palabras_train = palabras1 + palabras3

X_train, Y_train_reg = generar_dataset_multi(palabras_train, PALABRAS_OBJETIVO)

print("Tamaño de X_train:", X_train.shape)
for w in PALABRAS_OBJETIVO:
    print(f"Para '{w}', #ejemplos de conteo (Y_reg):", Y_train_reg[w].shape)

# ----------------------
# 6. Entrenar un modelo de regresión de conteo (Poisson) para cada palabra
# ----------------------
modelos_reg = {}
for w in PALABRAS_OBJETIVO:
    y_w = Y_train_reg[w]
    # Usamos PoissonRegressor, que asume etiquetas de conteo (enteras >= 0)
    reg = PoissonRegressor(max_iter=1000)
    reg.fit(X_train, y_w)
    modelos_reg[w] = reg
    print(f"✅ PoissonRegressor entrenado para palabra '{w}'")

# (Si quisieras un modelo lineal en vez de Poisson, reemplaza con LinearRegression)
# model_lin = LinearRegression(); model_lin.fit(X_train, y_w)

# ----------------------
# 7. Guardar los modelos entrenados a disco
# ----------------------
os.makedirs("modelos_edge", exist_ok=True)
for w, reg in modelos_reg.items():
    ruta = os.path.join("modelos_edge", f"{w}_poisson.pkl")
    with open(ruta, "wb") as f_out:
        pickle.dump(reg, f_out)
    print(f"Modelo (Poisson) para '{w}' guardado en: {ruta}")

# ----------------------
# 8. Función para simular “tiempo real” sobre trump2.txt (regresión de conteo->probabilidad)
# ----------------------
import math

def simular_tiempo_real_reg(palabras, modelos_reg, palabras_objetivo):
    """
    Recibe lista tokenizada de palabras (discurso), diccionario modelos_reg y lista de palabras_objetivo.
    Cada paso i devuelve un diccionario con:
      - índice actual
      - count_so_far[w] para cada w
      - lambda_w = conteo esperado de apariciones futuras (reg.predict)
      - probabilidad de al menos 1 aparición = 1 - exp(-lambda_w)
    """
    N = len(palabras)
    pos = posiciones_por_palabra(palabras, palabras_objetivo)

    # Inicializar contadores acumulados
    count_so_far = { w: 0 for w in palabras_objetivo }

    for i, w_actual in enumerate(palabras):
        # Conteos “lag” antes de procesar palabra actual
        count_until_prev = { w: count_so_far[w] for w in palabras_objetivo }

        # Construir feature_i
        freq_until_prev = {
            w: (count_until_prev[w] / (i + 1)) if (i + 1) > 0 else 0.0
            for w in palabras_objetivo
        }
        position_ratio = (i + 1) / N
        feature_i = [
            i,
            count_until_prev["china"],
            count_until_prev["economy"],
            count_until_prev["border"],
            freq_until_prev["china"],
            freq_until_prev["economy"],
            freq_until_prev["border"],
            position_ratio
        ]

        # Para cada palabra objetivo, predecir el conteo esperado (lambda) y luego p(≥1)
        resultados = {}
        for w in palabras_objetivo:
            reg = modelos_reg[w]
            # Predicción de conteo esperado (λ ≥ 0)
            lambda_w = max(0.0, reg.predict([feature_i])[0])
            # Probabilidad de aparecer al menos una vez: 1 - exp(-λ)
            prob_w = 1 - math.exp(-lambda_w)
            resultados[w] = {
                "lambda_esperado": float(lambda_w),
                "prob_al_menos_una": float(prob_w)
            }

        # Crear diccionario de salida
        salida = {
            "indice_actual": i,
            "palabra_actual": w_actual,
            "count_so_far": count_until_prev.copy(),
            "predicciones": resultados
        }

        print(salida)
        # (Podrías almacenar o enviar este diccionario en cada paso)

        # Actualizar count_so_far si la palabra actual es objetivo
        if w_actual in count_so_far:
            count_so_far[w_actual] += 1

    print("✅ Simulación regresión en tiempo real completada.")

# ----------------------
# 9. Ejecutar simulación sobre trump2.txt
# ----------------------
if __name__ == "__main__":
    palabras2 = cargar_palabras("trump2.txt")

    # Cargar modelos Poisson desde disco
    modelos_carg = {}
    for w in PALABRAS_OBJETIVO:
        ruta_modelo = os.path.join("modelos_edge", f"{w}_poisson.pkl")
        with open(ruta_modelo, "rb") as f_in:
            modelos_carg[w] = pickle.load(f_in)

    # Simulación “tiempo real” con regresión de conteo
    simular_tiempo_real_reg(palabras2, modelos_carg, PALABRAS_OBJETIVO)


Tamaño de X_train: (10522, 8)
Para 'china', #ejemplos de conteo (Y_reg): (10522,)
Para 'economy', #ejemplos de conteo (Y_reg): (10522,)
Para 'border', #ejemplos de conteo (Y_reg): (10522,)
✅ PoissonRegressor entrenado para palabra 'china'
✅ PoissonRegressor entrenado para palabra 'economy'
✅ PoissonRegressor entrenado para palabra 'border'
Modelo (Poisson) para 'china' guardado en: modelos_edge/china_poisson.pkl
Modelo (Poisson) para 'economy' guardado en: modelos_edge/economy_poisson.pkl
Modelo (Poisson) para 'border' guardado en: modelos_edge/border_poisson.pkl
{'indice_actual': 0, 'palabra_actual': 'thank', 'count_so_far': {'china': 0, 'economy': 0, 'border': 0}, 'predicciones': {'china': {'lambda_esperado': 10.210921104401043, 'prob_al_menos_una': 0.9999632334136327}, 'economy': {'lambda_esperado': 6.258987382991109, 'prob_al_menos_una': 0.9980868178631455}, 'border': {'lambda_esperado': 3.9346427705229363, 'prob_al_menos_una': 0.9804473171252854}}}
{'indice_actual': 1, 'palabra_ac

In [13]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
stats_edge.py

Este script recorre la transcripción trump2.txt palabra por palabra usando los modelos
de regresión de conteo (PoissonRegressor) entrenados para cada palabra objetivo,
midiendo latencia de inferencia, prediciendo λ (conteo esperado), calculando el conteo real
de apariciones futuras y, al final, reportando estadísticas (MAE, RMSE, latencia promedio).

Requisitos:
  - Python 3.7+
  - Paquetes instalados en el entorno virtual:
      pip install numpy scikit-learn
  - Archivo de transcripción: trump2.txt (lista de palabras tokenizadas, minúsculas).
  - Modelos entrenados en la carpeta modelos_edge/:
      modelos_edge/china_poisson.pkl
      modelos_edge/economy_poisson.pkl
      modelos_edge/border_poisson.pkl

Cómo usar:
  $ python stats_edge.py
"""

import os
import time
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

# -----------------------------
# 1. Configuración general
# -----------------------------
# Carpeta donde están los modelos Poisson para cada palabra objetivo
MODELOS_DIR = "modelos_edge"
# Ruta a la transcripción de prueba
TRANSCRIPCION_PATH = "trump2.txt"
# Lista de palabras objetivo
PALABRAS_OBJETIVO = ["china", "economy", "border"]

# Nombres de archivos de modelo para cada palabra objetivo
MODELO_ARCHIVOS = {
    "china":   os.path.join(MODELOS_DIR, "china_poisson.pkl"),
    "economy": os.path.join(MODELOS_DIR, "economy_poisson.pkl"),
    "border":  os.path.join(MODELOS_DIR, "border_poisson.pkl")
}


def cargar_palabras(ruta_txt):
    """
    Lee un archivo .txt completo, normaliza a minúsculas, elimina puntuación básica
    y retorna una lista de palabras tokenizadas.
    """
    with open(ruta_txt, "r", encoding="utf-8") as f:
        texto = f.read().lower()
    for ch in ['.', ',', ';', ':', '!', '?', '"', '“', '”', '(', ')', '[', ']', '—', '…']:
        texto = texto.replace(ch, "")
    palabras = texto.split()
    return palabras


def precalcular_posiciones(palabras, palabras_objetivo):
    """
    Dada la lista completa de palabras y la lista de palabras objetivo,
    retorna un diccionario {w: [idx1, idx2, ...]} con todas las posiciones
    donde aparece w en 'palabras'.
    """
    posiciones = {w: [] for w in palabras_objetivo}
    for idx, w in enumerate(palabras):
        if w in posiciones:
            posiciones[w].append(idx)
    return posiciones


def conteo_futuro_en_pos(posiciones, i):
    """
    Dada la estructura de 'posiciones' y un índice i, retorna un dict {w: count}
    con el conteo de apariciones de cada palabra w en posiciones > i.
    Las listas dentro de posiciones[w] están ya ordenadas creciente.
    """
    remaining = {}
    for w, lista_idx in posiciones.items():
        # contar cuántos índices en lista_idx son > i
        # (podemos usar np.searchsorted si es muy largo, pero aquí filtramos directamente)
        cnt = sum(1 for p in lista_idx if p > i)
        remaining[w] = cnt
    return remaining


def main():
    # -----------------------------
    # 2. Cargar transcripción de prueba
    # -----------------------------
    print("1) Cargando transcripción desde:", TRANSCRIPCION_PATH)
    palabras2 = cargar_palabras(TRANSCRIPCION_PATH)
    N = len(palabras2)
    print(f"   → Discurso de prueba tiene {N} palabras tokenizadas.\n")

    # -----------------------------
    # 3. Precalcular posiciones “futuras” de cada palabra objetivo
    # -----------------------------
    print("2) Precalculando posiciones de cada palabra objetivo en trump2.txt …")
    posiciones2 = precalcular_posiciones(palabras2, PALABRAS_OBJETIVO)

    # -----------------------------
    # 4. Cargar modelos Poisson desde disco
    # -----------------------------
    print("3) Cargando modelos Poisson para cada palabra objetivo …")
    modelos = {}
    for w in PALABRAS_OBJETIVO:
        ruta_modelo = MODELO_ARCHIVOS[w]
        if not os.path.isfile(ruta_modelo):
            raise FileNotFoundError(f"No se encontró el modelo para '{w}' en {ruta_modelo}")
        with open(ruta_modelo, "rb") as f_in:
            modelos[w] = pickle.load(f_in)
        print(f"   → Modelo cargado: {w} ← {ruta_modelo}")
    print()

    # -----------------------------
    # 5. Vectores para recopilar verdaderos conteos y predicciones
    # -----------------------------
    # Para cada palabra w, guardaremos:
    #   y_true[w]: lista de counts futuros reales (int) para cada posición i
    #   y_pred[w]: lista de lambda (float) predichos por el modelo
    y_true = {w: [] for w in PALABRAS_OBJETIVO}
    y_pred = {w: [] for w in PALABRAS_OBJETIVO}

    # Lista de latencias (ms) por token (usadas para comparar Edge vs Cloud)
    latencies_ms = []

    # Diccionario para conteos acumulados hasta el paso i (antes de procesar i)
    count_so_far = {w: 0 for w in PALABRAS_OBJETIVO}

    print("4) Iniciando simulación en tiempo real (Edge) …")
    # Recorrer cada palabra / índice i
    for i, w_actual in enumerate(palabras2):
        # 5.1. Contar apariciones futuras reales (y_true) antes de inferencia
        remaining_counts = conteo_futuro_en_pos(posiciones2, i)
        for w in PALABRAS_OBJETIVO:
            y_true[w].append(remaining_counts[w])

        # 5.2. Construir vector de features (igual que en entrenamiento)
        #   features = [i, count_so_far["china"], count_so_far["economy"], count_so_far["border"],
        #               freq_china, freq_economy, freq_border, position_ratio]
        total = N
        count_until_prev = {w: count_so_far[w] for w in PALABRAS_OBJETIVO}
        freq_until_prev = {
            w: (count_until_prev[w] / (i + 1)) if (i + 1) > 0 else 0.0
            for w in PALABRAS_OBJETIVO
        }
        position_ratio = (i + 1) / total

        feature_i = [
            i,
            count_until_prev["china"],
            count_until_prev["economy"],
            count_until_prev["border"],
            freq_until_prev["china"],
            freq_until_prev["economy"],
            freq_until_prev["border"],
            position_ratio
        ]
        feature_array = np.array(feature_i).reshape(1, -1)

        # 5.3. Inferencia: medir latencia y predecir lambda para cada palabra objetivo
        t0 = time.time()
        lambdas = {}
        for w in PALABRAS_OBJETIVO:
            reg = modelos[w]
            lambda_w = reg.predict(feature_array)[0]
            # Aseguramos no negativos (algún regresor puede dar < 0)
            lambda_w = float(lambda_w) if lambda_w > 0 else 0.0
            lambdas[w] = lambda_w
            y_pred[w].append(lambda_w)
        t1 = time.time()

        latency_ms = (t1 - t0) * 1000.0
        latencies_ms.append(latency_ms)

        # 5.4. Actualizar count_so_far para la siguiente iteración
        if w_actual in count_so_far:
            count_so_far[w_actual] += 1

    print("   → Simulación completada.\n")

    # -----------------------------
    # 6. Calcular métricas y presentar resultados
    # -----------------------------
    records = []
    for w in PALABRAS_OBJETIVO:
        true_arr = np.array(y_true[w])
        pred_arr = np.array(y_pred[w])

        mae = mean_absolute_error(true_arr, pred_arr)
        rmse = np.sqrt(mean_squared_error(true_arr, pred_arr))
        avg_latency = np.mean(latencies_ms)

        records.append({
            "Word": w,
            "MAE_Edge": mae,
            "RMSE_Edge": rmse,
            "AvgLatencyPerToken_Edge_ms": avg_latency
        })

    df_edge = pd.DataFrame(records)
    print("===== Resultados Estadísticos (Edge) =====")
    print(df_edge.to_string(index=False))

    # -----------------------------
    # 7. Guardar resultados a CSV (opcional)
    # -----------------------------
    salida_csv = "stats_edge_results.csv"
    df_edge.to_csv(salida_csv, index=False)
    print(f"\n   Resultados guardados en: {salida_csv}")


if __name__ == "__main__":
    main()


1) Cargando transcripción desde: trump2.txt
   → Discurso de prueba tiene 8703 palabras tokenizadas.

2) Precalculando posiciones de cada palabra objetivo en trump2.txt …
3) Cargando modelos Poisson para cada palabra objetivo …
   → Modelo cargado: china ← modelos_edge/china_poisson.pkl
   → Modelo cargado: economy ← modelos_edge/economy_poisson.pkl
   → Modelo cargado: border ← modelos_edge/border_poisson.pkl

4) Iniciando simulación en tiempo real (Edge) …
   → Simulación completada.

===== Resultados Estadísticos (Edge) =====
   Word  MAE_Edge  RMSE_Edge  AvgLatencyPerToken_Edge_ms
  china  4.506770   4.522212                    0.123291
economy  5.666859   5.672631                    0.123291
 border  2.453027   2.598312                    0.123291

   Resultados guardados en: stats_edge_results.csv


In [15]:
# 2.1 Instalar dependencies (solo la primera vez)
#!pip install vosk ffmpeg-python --quiet

# 2.2 Importar librerías necesarias
import subprocess
import time
import wave
from vosk import Model, KaldiRecognizer

def get_video_duration(path: str) -> float:
    """
    Devuelve la duración del video en segundos usando ffprobe.
    """
    try:
        result = subprocess.check_output([
            "ffprobe", "-v", "error",
            "-show_entries", "format=duration",
            "-of", "default=noprint_wrappers=1:nokey=1",
            path
        ], stderr=subprocess.STDOUT).decode().strip()
        return float(result)
    except Exception as e:
        raise RuntimeError(f"No se pudo obtener duración de '{path}': {e}")

def extract_audio_to_wav(video_path: str, wav_path: str):
    """
    Extrae audio del video (mono 16 kHz) a un archivo WAV temporal.
    """
    cmd = [
        "ffmpeg", "-y", "-i", video_path,
        "-ar", "16000", "-ac", "1", wav_path
    ]
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

def measure_vosk_latency(video_path: str, vosk_model_path: str, wav_path: str = "temp_audio.wav"):
    """
    Transcribe con Vosk (Edge) y mide:
      - t_total: tiempo total de transcripción (s)
      - dur_video: duración del video (s)
      - lat_s_por_s: latencia (s transcripción / s video)
    """
    # 1) Obtener duración del video
    duración = get_video_duration(video_path)

    # 2) Extraer audio a WAV
    print(f"🔊 Extrayendo audio de '{video_path}' a '{wav_path}' …")
    extract_audio_to_wav(video_path, wav_path)

    # 3) Cargar modelo Vosk (descarga un modelo en /content/vosk-model ...)
    print(f"📥 Cargando modelo Vosk desde '{vosk_model_path}' …")
    model = Model(vosk_model_path)
    rec = KaldiRecognizer(model, 16000)

    # 4) Abrir el WAV y transcribir
    wf = wave.open(wav_path, "rb")
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getframerate() != 16000:
        raise RuntimeError("WAV debe ser PCM mono 16 kHz")

    print(f"⏱ Transcribiendo '{wav_path}' con Vosk …")
    t0 = time.time()
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        rec.AcceptWaveform(data)
    t1 = time.time()
    wf.close()

    t_total = t1 - t0
    lat_s_por_s = t_total / duración if duración > 0 else float("nan")

    # 5) Eliminar el archivo WAV temporal
    subprocess.run(["rm", "-f", wav_path])

    return t_total, duración, lat_s_por_s

# 2.3 Ejecutar la medición para trump2.mp4 con un modelo Vosk descargado
video_file = "trump2.mp4"           # Asegúrate de que exista en Colab
vosk_model_folder = "vosk-model-small-en-us-0.15"    # Reemplaza con tu ruta al folder del modelo

# Ejemplo: si no has descargado el modelo, puedes hacerlo así (por ejemplo, modelo en inglés):
# !wget -qO- https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip | busybox unzip - >&/dev/null
# vosk_model_folder = "vosk-model-small-en-us-0.15"

t_total_edge, dur_video_edge, lat_s_por_s_edge = measure_vosk_latency(video_file, vosk_model_folder)

print("\n=== Resultados Vosk (Edge) ===")
print(f"Latencia total       : {t_total_edge:.2f} s")
print(f"Duración del video   : {dur_video_edge:.2f} s")
print(f"Latencia por segundo : {lat_s_por_s_edge:.4f} s/s")


🔊 Extrayendo audio de 'trump2.mp4' a 'temp_audio.wav' …
📥 Cargando modelo Vosk desde 'vosk-model-small-en-us-0.15' …


LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=10 max-active=3000 lattice-beam=2
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from vosk-model-small-en-us-0.15/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from vosk-model-small-en-us-0.15/graph/HCLr.fst vosk-model-small-en-us-0.15/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:303) Loading winfo vosk-model-small-en-us-0.15/graph/phones/word_boundary.int


⏱ Transcribiendo 'temp_audio.wav' con Vosk …

=== Resultados Vosk (Edge) ===
Latencia total       : 125.74 s
Duración del video   : 3498.68 s
Latencia por segundo : 0.0359 s/s


In [16]:
with open('trump2.txt', 'r') as file:
    data = file.read()
    print(len(data.split()))  # Imprime el número de palabras en el archivo

8703
